In [ ]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch

: 

In [ ]:
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'

model = BertForQuestionAnswering.from_pretrained(model_name)

tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
sunset_motors_context = """
Sunset Motors was established in 2005 and is located in Crestwood.
The dealership covers an area of 10 acres and specializes in new and used vehicles.
Sunset Motors offers cars from manufacturers such as Ford, Toyota, Honda, Chevrolet, and BMW.
The company is known for excellent customer service and competitive pricing.
"""


In [ ]:
def faq_bot(question):
    encoding = tokenizer.encode_plus(
        question,
        sunset_motors_context,
        add_special_tokens=True,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"]
    token_type_ids = encoding["token_type_ids"]
    attention_mask = encoding["attention_mask"]

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )

    start_index = torch.argmax(outputs.start_logits[0]).item()
    end_index = torch.argmax(outputs.end_logits[0]).item() + 1

    if end_index <= start_index:
        return "I'm unable to find the answer to this question."

    answer_tokens = tokens[start_index:end_index]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    # Clean up subword tokens (##)
    cleaned_answer = ""
    for word in answer.split():
        if word.startswith("##"):
            cleaned_answer += word[2:]
        else:
            cleaned_answer += " " + word

    return cleaned_answer.strip()

In [ ]:
print(faq_bot("Where is the dealership located?"))
print(faq_bot("What make of cars are available?"))
print(faq_bot("How large is the dealership?"))
